In [238]:
import requests
import json

# Example ticket-tagging-api App usage.

This notebook shows how to use the ticket-tagging-api app.  It does not use the Steamship SDK to make it easier to port to other languages.

Each instance of the app has its own built-in context where examples are collected, labels are stored, etc.

## Prerequisites

* Using this assumes that you have already created an instance of the app (this is shown with the SDK in the app_instance_creation notebook.
* You must have the api_key associated with the deployed app instance.



In [239]:
#The base_url for the Steamship app, from the other notebook
base_url = 'http://host.docker.internal:8081/@dev/ticket-tagging-api-20220613/ticket-tagging-api-20220613/'

In [240]:
# Your Steamship API key.
api_key = "CHANGEME"

In [241]:
headers = {"Authorization": f"Bearer {api_key}"}

In [242]:
# Tiny wrapper for post calls of json data to the app.
def app_call(route: str, parameters: dict):
    return requests.post(base_url + route, headers=headers, json=parameters)

## First - a toy example for demonstrating the basic API

In [243]:
#clear any previous examples
app_call('clear_examples',{}).text

'Removed 1908 examples'

In [244]:
# Set the labels we'll use
response = app_call('set_labels',{'labels':['lion','tiger','elephant']})
response.text

'Labels accepted'

In [246]:
# Try a ticket classification (no training yet, so this is zero-shot)
response = app_call('tag_ticket',{'ticket_text':'I like big cats'})
response.json()

{'tiger': 0.6951355934143066,
 'lion': 0.6889989972114563,
 'elephant': 0.1538180708885193}

In [247]:
# Create some example tickets
examples = [('This is a ticket about lions.', ['lion']), ('This is a ticket about tigers.',['tiger']), ('This is a ticket about an elephant.',['elephant'])]

In [248]:
# Push the examples to the app
for example in examples:
    response = app_call('add_example', {'ticket_text':example[0], 'labels':example[1]})
    print(response.text)

Example accepted
Example accepted
Example accepted


In [249]:
# How many examples have we added?
response = app_call('count_examples', {})
response.json()

{'label_examples': {'tiger': 1, 'lion': 1, 'elephant': 1}, 'total_examples': 3}

In [250]:
# We haven't specialized our model yet, so this should return UNSPECIALIZED
response = app_call('specialize_status', {})
response.json()

{'status': 'UNSPECIALIZED'}

In [251]:
# Remove the three toy examples
app_call('clear_examples',{}).text

'Removed 3 examples'

## Now, let's attempt with real data

In [252]:
# Read data adapted from test dataset, stripped to just the text to be tagged and lists of 'cleaned' tags (no underscores or leading or trailing spaces)
with open('test_data.json') as data_file:
    data = json.load(data_file)
    
len(data)

2133

In [253]:
# Find all unique tag labels
labels = list(set([label for row in data for label in row['tags']]))

In [254]:
# Set the app's labels
response = app_call('set_labels',{'labels':labels})
response.text

'Labels accepted'

In [255]:
# Tag an example ticket.  We haven't specialized yet, so this is zero-shot
response = app_call('tag_ticket',{'ticket_text':data[0]['text']})
response.json()

{'alerts acknowledged': 0.9112120270729065,
 'login': 0.5004827976226807,
 'redacted content': 0.07734347134828568,
 'realtime overview': 0.21536551415920258,
 'how do i': 0.7680781483650208,
 'integratons': 0.7087875008583069,
 'scheduling': 0.06945476680994034,
 'spam': 0.003957747947424652,
 'people': 0.7233816385269165,
 'n/a': 0.049051716923713684,
 'ticket mapping/queues': 0.07419610023498532,
 'self-served': 0.0029283713083714247,
 'automations': 0.10863953828811646,
 'needs review': 0.988425076007843,
 'closed by merge': 0.00860019400715828,
 'performance': 0.8535534143447876,
 'security': 0.23934194445610046,
 'api': 0.8333168625831604,
 'web widget': 0.248447448015213,
 'bug': 0.5133253335952759,
 'assembled action needed': 0.9657626152038574,
 'feature request': 0.8226450681686401,
 'alerts': 0.5230124592781067,
 'inbound sales prospect': 0.24141883850097656,
 'why isn t this working': 0.6911829113960266,
 'gcal': 0.6264333724975586,
 'reports': 0.5989434719085693,
 'forecas

## Next lets specialize the model

There were not enough examples of all of the ticket types in the dataset, so we'll demonstrate here with only the top 10 ticket types.

In [256]:
top_10_tags = ['how do i',
 'n/a',
 'why isn t this working',
 'not a support issue',
 'datadog alerts support',
 'datadog alerts',
 'forecasting and staffing',
 'scheduling',
 'inbound sales prospect',
 'people']

In [257]:
# Set the labels in the app
response = app_call('set_labels',{'labels':top_10_tags})
response.text

'Labels accepted'

In [258]:
# Create dataset with only the top tags
top_tag_data = [dict(text=row['text'], tags=[tag for tag in row['tags'] if tag in top_10_tags]) for row in data] 

In [259]:
# Add examples to the app
for row in top_tag_data:
    response = app_call('add_example', {'ticket_text':row['text'], 'labels':row['tags']})
    

In [260]:
# Count the total examples per ticket.  Note the total here is less than the total in the dataset, because after filtering to the top 10 tags, some examples don't have any tags anymore.
response = app_call('count_examples', {})
response.json()

{'label_examples': {'people': 188,
  'not a support issue': 255,
  'inbound sales prospect': 191,
  'datadog alerts support': 194,
  'forecasting and staffing': 191,
  'why isn t this working': 386,
  'scheduling': 191,
  'how do i': 716,
  'n/a': 450,
  'datadog alerts': 194},
 'total_examples': 1908}

In [261]:
#Start the specialization (training) process
response = app_call('start_specialize',{})
response.text

'Started specialization. Please poll specialize_status'

In [262]:
# Check with the app and make sure training started successfully.
response = app_call('specialize_status', {})
response.json()

{'status': 'SPECIALIZATION_IN_PROGRESS'}

In [263]:
# Can still tag tickets with zero-shot while waiting
response = app_call('tag_ticket',{'ticket_text':'Another example ticket about Adherence'})
response.json()

{'not a support issue': 0.0013767160708084702,
 'datadog alerts': 0.0012334269704297185,
 'people': 0.007066965103149414,
 'datadog alerts support': 0.0009663285454735157,
 'forecasting and staffing': 0.0004400350444484502,
 'why isn t this working': 0.009248231537640098,
 'scheduling': 0.0002453009074088185,
 'how do i': 0.22574004530906677,
 'inbound sales prospect': 0.010559393092989922,
 'n/a': 0.0007463498623110354}

In [268]:
# Now need to wait ~5 hours, or poll continuously
response = app_call('specialize_status', {})
response.json()

{'status': 'SPECIALIZED'}

In [269]:
# Tag an example ticket.  We've specialized, so now we're using the trained model under the hood.
response = app_call('tag_ticket',{'ticket_text':data[0]['text']})
response.json()

{'not a support issue': 0.013251291587948804,
 'people': 0.024483805522322655,
 'how do i': 0.2636350095272064,
 'datadog alerts support': 0.00026704734773375083,
 'forecasting and staffing': 0.057214416563510916,
 'why isn t this working': 0.19870997965335846,
 'scheduling': 0.01933469437062741,
 'datadog alerts': 0.00033931879443116497,
 'inbound sales prospect': 0.008816328831017017,
 'n/a': 0.028928974643349648}